In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\mnuri\\OneDrive\\Documents\\Data Analysis with Python\\Project 2 User Segmentation'

In [3]:
df = pd.read_csv('online_retail.csv', header=0)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df_clean = df.copy()
# konversi invoiceDate jadi datetime
df_clean ['InvoiceDate'] = df_clean['InvoiceDate'].astype('datetime64[ns]')
# kolom year month
df_clean ['Year Month'] = df_clean ['InvoiceDate'].dt.to_period('M')
# hapus semua baris tanpa kolom customer id
df_clean = df_clean[~df_clean['CustomerID'].isna()]
# hapus semua baris tanpa kolom description
df_clean = df_clean[~df_clean['Description'].isna()]
# mengubah semua huruf description menjadi kecil
df_clean['Description'] = df_clean['Description'].str.lower()
# membuat kolom order status dengan nilai "cencelled" jika InvoiceNo diawali dengan huruf 'c' dan 'delivered' jika InvoiceNo tanpa awalan huruf 'c'
df_clean ['Order Status'] = np.where(df_clean['InvoiceNo'].str[:1]=='C', 'cancelled', 'delivered')
# menghapus baris dengan unitprice bernial negatif
df_clean = df_clean[df_clean['UnitPrice']>0]
# membuat nilai amount dengan perkalian antara quantity dan unitprice
df_clean['Amount'] = df_clean['Quantity'] * df_clean['UnitPrice']
# mengganti description dari stockcode yang memiliki beberapa description dengan salah satu description-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['StockCode','Description'], as_index=False).agg(order_cnt=('InvoiceNo','nunique')).sort_values(['StockCode','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('StockCode')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'Description':'most_freq_Description'}), how='left', on='StockCode')
df_clean['Description'] = df_clean['most_freq_Description']
df_clean = df_clean.drop(columns='most_freq_Description')
# mengkonversi customer_id menjadi string
df_clean['CustomerID'] = df_clean['CustomerID'].astype(str)
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['Quantity','Amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year Month,Order Status,Amount
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12,delivered,15.30
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,delivered,20.34
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12,delivered,22.00
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,delivered,20.34
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,delivered,20.34
...,...,...,...,...,...,...,...,...,...,...,...
406412,581587,22613,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12,delivered,10.20
406413,581587,22899,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12,delivered,12.60
406414,581587,23254,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12,delivered,16.60
406415,581587,23255,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12,delivered,16.60


In [6]:
# object to int
df['InvoiceNo'] = pd.to_numeric(df['InvoiceNo'], errors='coerce')
df['InvoiceNo'] = df['InvoiceNo'].fillna(-1).astype('int')

# float to int
df['UnitPrice'] = df['UnitPrice'].astype('int')
df['CustomerID'] = df['CustomerID'].astype('Int64')

# object to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  int64         
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  int64         
 6   CustomerID   406829 non-null  Int64         
 7   Country      541909 non-null  object        
dtypes: Int64(1), datetime64[ns](1), int64(3), object(3)
memory usage: 33.6+ MB


In [8]:
df_clean['Amount']

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
406412    10.20
406413    12.60
406414    16.60
406415    16.60
406416    14.85
Name: Amount, Length: 406417, dtype: float64

In [9]:
df_user = df_clean.groupby('CustomerID', as_index=False).agg(order_cnt=('InvoiceNo','nunique'), max_order_date=('InvoiceDate','max'),total_order_value=('Amount','sum'))
df_user

,CustomerID,order_cnt,max_order_date,total_order_value
0,12347.0,7,2011-12-07 15:52:00,4310.00
1,12348.0,4,2011-09-25 13:13:00,1797.24
2,12349.0,1,2011-11-21 09:51:00,1757.55
3,12350.0,1,2011-02-02 16:01:00,334.40
4,12352.0,11,2011-11-03 14:37:00,1545.41
...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60
4356,18281.0,1,2011-06-12 10:53:00,80.82
4357,18282.0,3,2011-12-02 11:43:00,176.60
4358,18283.0,16,2011-12-06 12:02:00,2094.88


In [12]:
today

Timestamp('2011-12-09 12:50:00')

In [11]:
today = df_clean['InvoiceDate'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,CustomerID,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12347.0,7,2011-12-07 15:52:00,4310.00,1
1,12348.0,4,2011-09-25 13:13:00,1797.24,74
2,12349.0,1,2011-11-21 09:51:00,1757.55,18
3,12350.0,1,2011-02-02 16:01:00,334.40,309
4,12352.0,11,2011-11-03 14:37:00,1545.41,35
...,...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60,277
4356,18281.0,1,2011-06-12 10:53:00,80.82,180
4357,18282.0,3,2011-12-02 11:43:00,176.60,7
4358,18283.0,16,2011-12-06 12:02:00,2094.88,3


In [13]:
df_user.describe()

,order_cnt,max_order_date,total_order_value,day_since_last_order
count,4360.000000,4360,4360.000000,4360.000000
mean,5.061927,2011-09-08 23:24:16.844036608,1803.309561,91.038532
min,1.000000,2010-12-01 09:53:00,-1192.200000,0.000000
25%,1.000000,2011-07-20 05:33:45,292.335000,16.000000
50%,3.000000,2011-10-20 15:56:30,644.475000,49.000000
75%,5.000000,2011-11-23 09:12:15,1600.442500,142.000000
max,244.000000,2011-12-09 12:50:00,268174.460000,373.000000
std,9.259052,NaN,6895.377077,100.725753


In [14]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,CustomerID,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score
0,12347.0,7,2011-12-07 15:52:00,4310.00,1,5
1,12348.0,4,2011-09-25 13:13:00,1797.24,74,2
2,12349.0,1,2011-11-21 09:51:00,1757.55,18,4
3,12350.0,1,2011-02-02 16:01:00,334.40,309,1
4,12352.0,11,2011-11-03 14:37:00,1545.41,35,3
...,...,...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60,277,1
4356,18281.0,1,2011-06-12 10:53:00,80.82,180,1
4357,18282.0,3,2011-12-02 11:43:00,176.60,7,5
4358,18283.0,16,2011-12-06 12:02:00,2094.88,3,5


In [15]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CustomerID            4360 non-null   object        
 1   order_cnt             4360 non-null   int64         
 2   max_order_date        4360 non-null   datetime64[ns]
 3   total_order_value     4360 non-null   float64       
 4   day_since_last_order  4360 non-null   int64         
 5   recency_score         4360 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 204.5+ KB


In [16]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[0,
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,CustomerID,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12347.0,7,2011-12-07 15:52:00,4310.00,1,5,4
1,12348.0,4,2011-09-25 13:13:00,1797.24,74,2,3
2,12349.0,1,2011-11-21 09:51:00,1757.55,18,4,1
3,12350.0,1,2011-02-02 16:01:00,334.40,309,1,1
4,12352.0,11,2011-11-03 14:37:00,1545.41,35,3,5
...,...,...,...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60,277,1,1
4356,18281.0,1,2011-06-12 10:53:00,80.82,180,1,1
4357,18282.0,3,2011-12-02 11:43:00,176.60,7,5,3
4358,18283.0,16,2011-12-06 12:02:00,2094.88,3,5,5


In [17]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CustomerID            4360 non-null   object        
 1   order_cnt             4360 non-null   int64         
 2   max_order_date        4360 non-null   datetime64[ns]
 3   total_order_value     4360 non-null   float64       
 4   day_since_last_order  4360 non-null   int64         
 5   recency_score         4360 non-null   int64         
 6   frequency_score       4360 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 238.6+ KB


In [18]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,CustomerID,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12347.0,7,2011-12-07 15:52:00,4310.00,1,5,4,5
1,12348.0,4,2011-09-25 13:13:00,1797.24,74,2,3,4
2,12349.0,1,2011-11-21 09:51:00,1757.55,18,4,1,4
3,12350.0,1,2011-02-02 16:01:00,334.40,309,1,1,2
4,12352.0,11,2011-11-03 14:37:00,1545.41,35,3,5,4
...,...,...,...,...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60,277,1,1,1
4356,18281.0,1,2011-06-12 10:53:00,80.82,180,1,1,1
4357,18282.0,3,2011-12-02 11:43:00,176.60,7,5,3,1
4358,18283.0,16,2011-12-06 12:02:00,2094.88,3,5,5,5


In [19]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CustomerID            4360 non-null   object        
 1   order_cnt             4360 non-null   int64         
 2   max_order_date        4360 non-null   datetime64[ns]
 3   total_order_value     4360 non-null   float64       
 4   day_since_last_order  4360 non-null   int64         
 5   recency_score         4360 non-null   int64         
 6   frequency_score       4360 non-null   int64         
 7   monetary_score        4360 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 272.6+ KB


In [20]:
import numpy as np

df_user['segment'] = np.select(
    [(df_user['recency_score'] == 5) & (df_user['frequency_score'] >= 4),
        (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score'] >= 4),
        (df_user['recency_score'] >= 4) & (df_user['frequency_score'].between(2, 3)),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'] == 5),
        (df_user['recency_score'] == 3) & (df_user['frequency_score'] == 3),
        (df_user['recency_score'] == 5) & (df_user['frequency_score'] == 1),
        (df_user['recency_score'] == 4) & (df_user['frequency_score'] == 1),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'].between(3, 4)),
        (df_user['recency_score'] == 3) & (df_user['frequency_score'] <= 2),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'] <= 2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists',"04-Can't Lose Them", '05-Need Attention', '06-New Customers','07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating'],
    default='Unknown'  # Optional default value if no condition matches
)
df_user

,CustomerID,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score,segment
0,12347.0,7,2011-12-07 15:52:00,4310.00,1,5,4,5,01-Champion
1,12348.0,4,2011-09-25 13:13:00,1797.24,74,2,3,4,08-At Risk
2,12349.0,1,2011-11-21 09:51:00,1757.55,18,4,1,4,07-Promising
3,12350.0,1,2011-02-02 16:01:00,334.40,309,1,1,2,10-Hibernating
4,12352.0,11,2011-11-03 14:37:00,1545.41,35,3,5,4,02-Loyal Customers
...,...,...,...,...,...,...,...,...,...
4355,18280.0,1,2011-03-07 09:52:00,180.60,277,1,1,1,10-Hibernating
4356,18281.0,1,2011-06-12 10:53:00,80.82,180,1,1,1,10-Hibernating
4357,18282.0,3,2011-12-02 11:43:00,176.60,7,5,3,1,03-Potential Loyalists
4358,18283.0,16,2011-12-06 12:02:00,2094.88,3,5,5,5,01-Champion


In [21]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CustomerID            4360 non-null   object        
 1   order_cnt             4360 non-null   int64         
 2   max_order_date        4360 non-null   datetime64[ns]
 3   total_order_value     4360 non-null   float64       
 4   day_since_last_order  4360 non-null   int64         
 5   recency_score         4360 non-null   int64         
 6   frequency_score       4360 non-null   int64         
 7   monetary_score        4360 non-null   int64         
 8   segment               4360 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 306.7+ KB


In [24]:
summary = pd.pivot_table(df_user, index='segment',
               values=['CustomerID','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'CustomerID': pd.Series.nunique,
                        'day_since_last_order': [np.mean, np.median],
                        'order_cnt': [np.mean, np.median],
                        'total_order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['CustomerID'] / summary['CustomerID'].sum() * 100).round(1)
summary

CustomerID day_since_last_order         order_cnt  \
                          nunique                 mean median       mean   
segment                                                                    
01-Champion                   559             4.209302    3.0  16.300537   
02-Loyal Customers            632            30.550633   27.0   9.148734   
03-Potential Loyalists        607            14.413509   15.0   2.912685   
04-Can't Lose Them             51           140.607843  108.0  10.784314   
05-Need Attention             189            49.116402   49.0   3.402116   
06-New Customers               59             5.661017    6.0   1.000000   
07-Promising                  153            22.098039   23.0   1.000000   
08-At Risk                    436           143.878440  126.0   4.027523   
09-About to Sleep             432            50.768519   51.0   1.402778   
10-Hibernating               1242           210.697262  211.0   1.321256   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              10.0       6863.832576  2916.170       12.8  
02-Loyal Customers        7.0       3095.411267  2042.935       14.5  
03-Potential Loyalists    3.0        842.528962   652.740       13.9  
04-Can't Lose Them        9.0       2688.557255  2111.080        1.2  
05-Need Attention         3.0        993.448095   810.500        4.3  
06-New Customers          1.0        367.610339   244.410        1.4  
07-Promising              1.0        300.912157   223.880        3.5  
08-At Risk                4.0       1108.656725   802.170       10.0  
09-About to Sleep         1.0        455.584606   339.820        9.9  
10-Hibernating            1.0        390.530363   288.920       28.5